# [Redis](https://python.langchain.com/v0.2/docs/integrations/providers/redis/)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

## Cache
The Cache wrapper allows for Redis to be used as a remote, low-latency, in-memory cache for LLM prompts and responses.

Standard Cache
The standard cache is the Redis bread & butter of use case in production for both open-source and enterprise users globally.

In [ ]:
from langchain.cache import RedisCache

In [ ]:
redis_url = "redis://:@localhost:6379/0" # I do"n't set username and password

In [ ]:
from langchain.globals import set_llm_cache
import redis

redis_client = redis.Redis.from_url(redis_url)
set_llm_cache(RedisCache(redis_client))

In [ ]:
%%time
llm.invoke("Tell me a joke")

In [ ]:
%%time
llm.invoke("Tell me a joke")

In [ ]:
%%time
llm.invoke("Tell me why sky is blue?")

In [ ]:
%%time
llm.invoke("Tell me why sky is blue?")

## Semantic Cache
Semantic caching allows users to retrieve cached prompts based on semantic similarity between the user input and previously cached results. Under the hood it blends Redis as both a cache and a vectorstore.

In [ ]:
from langchain.cache import RedisSemanticCache
from langchain_openai import OpenAIEmbeddings

from langchain.globals import set_llm_cache
import redis

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)
# must use DB0
redis_url = "redis://:@localhost:6379/0" # I do"n't set username and password

set_llm_cache(RedisSemanticCache(
    embedding=embeddings,
    redis_url=redis_url
))

In [ ]:
%%time
# 第一次调用时，尚未缓存，所以需要更长的时间
llm.invoke("Tell me a joke")

In [ ]:
%%time
# 第二次调用时，虽然不是直接命中，但问题在语义上与原始问题相似，
# 因此使用了缓存的结果！
llm.invoke("Tell me one joke")